In [38]:
import uuid

In [317]:
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver import Chrome
from selenium import webdriver
import re
import time
import pandas as pd
import datetime
from tqdm import tqdm
from datetime import date
from selenium.webdriver.chrome.options import Options

main_url = 'http://cronica.diputados.gob.mx:8080/exist/siid2/siid2.xql?legis=LXII#'

options = Options()
options.add_argument('--headless')
driver = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183
[WDM] - Driver [/Users/cblanesg/.wdm/drivers/chromedriver/mac64/85.0.4183.87/chromedriver] found in cache


## Legislatura 64

In [318]:
main_url = 'http://sitl.diputados.gob.mx/LXIV_leg/listado_diputados_gpnp.php?tipot=TOTAL'
driver.get(main_url)

In [297]:
links_diputados = []
names_diputados = []
for i in tqdm(driver.find_elements_by_css_selector('a')):
    link = i.get_attribute('href')
    if any(re.findall('curricula', link)):
        links_diputados.append(link)
        names_diputados.append(re.sub('^[0-9]+ ', '', i.text))
        
    else:
        continue

100%|██████████| 511/511 [00:16<00:00, 30.31it/s]


In [298]:
mapping_name_party = {}
for i in names_diputados[0:251]:
    mapping_name_party[i] = 'MORENA'
    
for i in names_diputados[251: 329]:
    mapping_name_party[i] = 'PAN'
    
for i in names_diputados[329:377]:
    mapping_name_party[i] = 'PRI'
    
for i in names_diputados[377:423]:
    mapping_name_party[i] = 'PT'
    
for i in names_diputados[423:450]:
    mapping_name_party[i] = 'MC'
    
for i in names_diputados[450:474]:
    mapping_name_party[i] = 'ES'
    
for i in names_diputados[474:486]:
    mapping_name_party[i] = 'PRD'
    
for i in names_diputados[486:497]:
    mapping_name_party[i] = 'PVEM'

for i in names_diputados[497:]:
    mapping_name_party[i] = 'NA'

In [299]:
len(mapping_name_party)

500

In [303]:
def obtain_id_data(name, mapping_parties, link):
    suplente_propietario = unidecode.unidecode(driver.find_element_by_css_selector('body > div > table.cajasombra > tbody > tr > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(6) > td:nth-child(2)').text)
    
    if any(re.findall('ficha curricular', suplente_propietario)):
        dict_id = {}
        dict_id['legislatura'] = 64
        dict_id['id_legislador'] = str(uuid.uuid1())
        dict_id['nombre_completo'] = re.sub('[ ]+$', '', re.sub('[(]ficha curricular[)]', '', suplente_propietario))
        dict_id['tipo_legislador'] = unidecode.unidecode(driver.find_element_by_css_selector('body > div > table.cajasombra > tbody > tr > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(2) > td:nth-child(2) > strong').text)
        dict_id['entidad'] = unidecode.unidecode(driver.find_element_by_css_selector('body > div > table.cajasombra > tbody > tr > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(3) > td:nth-child(2) > strong:nth-child(1)').text)
        dict_id['distrito'] = driver.find_element_by_css_selector('body > div > table.cajasombra > tbody > tr > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(3) > td:nth-child(2) > strong:nth-child(3)').text
        dict_id['curul'] = driver.find_element_by_css_selector('body > div > table.cajasombra > tbody > tr > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(3) > td:nth-child(2) > strong:nth-child(5)').text
        dict_id['suplente_propietario'] = 'P'
        dict_id['name_suplente'] = unidecode.unidecode(driver.find_element_by_css_selector('body > div > table.cajasombra > tbody > tr > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(6) > td:nth-child(2)').text)
        dict_id['partido'] = mapping_parties[name]
        dict_id['link_curricula'] = link
        
        
        dict_sup = {}
        dict_sup['legislatura'] = 64
        dict_sup['id_legislador'] = str(uuid.uuid1())
        dict_sup['nombre_completo'] = name
        dict_sup['tipo_legislador'] = unidecode.unidecode(driver.find_element_by_css_selector('body > div > table.cajasombra > tbody > tr > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(2) > td:nth-child(2) > strong').text)
        dict_sup['entidad'] = unidecode.unidecode(driver.find_element_by_css_selector('body > div > table.cajasombra > tbody > tr > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(3) > td:nth-child(2) > strong:nth-child(1)').text)
        dict_sup['distrito'] = driver.find_element_by_css_selector('body > div > table.cajasombra > tbody > tr > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(3) > td:nth-child(2) > strong:nth-child(3)').text
        dict_sup['curul'] = driver.find_element_by_css_selector('body > div > table.cajasombra > tbody > tr > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(3) > td:nth-child(2) > strong:nth-child(5)').text
        dict_sup['suplente_propietario'] = 'S'
        dict_sup['name_suplente'] = ''
        dict_sup['partido'] = mapping_parties[name]
        dict_sup['link_curricula'] = link
        return([dict_id, dict_sup])
    else:
        dict_id = {}
        dict_id['legislatura'] = 64
        dict_id['id_legislador'] = str(uuid.uuid1())
        dict_id['nombre_completo'] = name
        dict_id['tipo_legislador'] = unidecode.unidecode(driver.find_element_by_css_selector('body > div > table.cajasombra > tbody > tr > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(2) > td:nth-child(2) > strong').text)
        dict_id['entidad'] = unidecode.unidecode(driver.find_element_by_css_selector('body > div > table.cajasombra > tbody > tr > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(3) > td:nth-child(2) > strong:nth-child(1)').text)
        dict_id['distrito'] = driver.find_element_by_css_selector('body > div > table.cajasombra > tbody > tr > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(3) > td:nth-child(2) > strong:nth-child(3)').text
        dict_id['curul'] = driver.find_element_by_css_selector('body > div > table.cajasombra > tbody > tr > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(3) > td:nth-child(2) > strong:nth-child(5)').text
        dict_id['suplente_propietario'] = 'P'
        dict_id['name_suplente'] = suplente_propietario
        dict_id['partido'] = mapping_parties[name]
        dict_id['link_curricula'] = link
        
        
        dict_sup = {}
        dict_sup['legislatura'] = 64
        dict_sup['id_legislador'] = str(uuid.uuid1())
        dict_sup['nombre_completo'] = suplente_propietario
        dict_sup['tipo_legislador'] = unidecode.unidecode(driver.find_element_by_css_selector('body > div > table.cajasombra > tbody > tr > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(2) > td:nth-child(2) > strong').text)
        dict_sup['entidad'] = unidecode.unidecode(driver.find_element_by_css_selector('body > div > table.cajasombra > tbody > tr > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(3) > td:nth-child(2) > strong:nth-child(1)').text)
        dict_sup['distrito'] = driver.find_element_by_css_selector('body > div > table.cajasombra > tbody > tr > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(3) > td:nth-child(2) > strong:nth-child(3)').text
        dict_sup['curul'] = driver.find_element_by_css_selector('body > div > table.cajasombra > tbody > tr > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(3) > td:nth-child(2) > strong:nth-child(5)').text
        dict_sup['suplente_propietario'] = 'S'
        dict_sup['name_suplente'] = ''
        dict_sup['partido'] = mapping_parties[name]
        dict_sup['link_curricula'] = link
        return([dict_id, dict_sup])

In [301]:
def obtain_id_data63(name, mapping_parties, link):
    suplente_propietario = driver.find_element_by_css_selector('body > table:nth-child(1) > tbody > tr:nth-child(3) > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(6) > td:nth-child(2) > strong > font').text
    if any(re.findall('ficha curricular', suplente_propietario)):
        dict_prop = {}
        dict_prop['legislatura'] = 63
        dict_prop['id_legislador'] = str(uuid.uuid1())
        dict_prop['nombre_completo'] = re.sub('[ ]+$', '', re.sub('[(]ficha curricular[)]', '', suplente_propietario))
        dict_prop['tipo_legislador'] = driver.find_element_by_css_selector('body > table:nth-child(1) > tbody > tr:nth-child(3) > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(2) > td:nth-child(2) > strong > font').text
        dict_prop['entidad'] = driver.find_element_by_css_selector('body > table:nth-child(1) > tbody > tr:nth-child(3) > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(3) > td:nth-child(2) > strong:nth-child(1) > font').text
        dict_prop['distrito'] = driver.find_element_by_css_selector('body > table:nth-child(1) > tbody > tr:nth-child(3) > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(3) > td:nth-child(2) > strong:nth-child(4) > font').text
        dict_prop['curul'] = driver.find_element_by_css_selector('body > table:nth-child(1) > tbody > tr:nth-child(3) > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(3) > td:nth-child(2) > strong:nth-child(7) > font').text
        dict_prop['suplente_propietario'] = 'P'
        dict_prop['name_suplente'] = name
        dict_prop['partido'] = mapping_parties[name]
        dict_prop['link_curricula'] = link
        
        
        dict_sup = {}
        dict_sup['legislatura'] = 63
        dict_sup['id_legislador'] = str(uuid.uuid1())
        dict_sup['nombre_completo'] = name
        dict_sup['tipo_legislador'] = driver.find_element_by_css_selector('body > table:nth-child(1) > tbody > tr:nth-child(3) > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(2) > td:nth-child(2) > strong > font').text
        dict_sup['entidad'] = driver.find_element_by_css_selector('body > table:nth-child(1) > tbody > tr:nth-child(3) > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(3) > td:nth-child(2) > strong:nth-child(1) > font').text
        dict_sup['distrito'] = driver.find_element_by_css_selector('body > table:nth-child(1) > tbody > tr:nth-child(3) > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(3) > td:nth-child(2) > strong:nth-child(4) > font').text
        dict_sup['curul'] = driver.find_element_by_css_selector('body > table:nth-child(1) > tbody > tr:nth-child(3) > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(3) > td:nth-child(2) > strong:nth-child(7) > font').text
        dict_sup['suplente_propietario'] = 'S'
        dict_sup['name_suplente'] = ''
        dict_sup['partido'] = mapping_parties[name]
        dict_sup['link_curricula'] = link
        return([dict_prop, dict_sup])
    else:
    
        dict_prop = {}
        dict_prop['legislatura'] = 63
        dict_prop['id_legislador'] = str(uuid.uuid1())
        dict_prop['nombre_completo'] = name
        dict_prop['tipo_legislador'] = driver.find_element_by_css_selector('body > table:nth-child(1) > tbody > tr:nth-child(3) > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(2) > td:nth-child(2) > strong > font').text
        dict_prop['entidad'] = driver.find_element_by_css_selector('body > table:nth-child(1) > tbody > tr:nth-child(3) > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(3) > td:nth-child(2) > strong:nth-child(1) > font').text
        dict_prop['distrito'] = driver.find_element_by_css_selector('body > table:nth-child(1) > tbody > tr:nth-child(3) > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(3) > td:nth-child(2) > strong:nth-child(4) > font').text
        dict_prop['curul'] = driver.find_element_by_css_selector('body > table:nth-child(1) > tbody > tr:nth-child(3) > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(3) > td:nth-child(2) > strong:nth-child(7) > font').text
        dict_prop['suplente_propietario'] = 'P'
        dict_prop['name_suplente'] = suplente_propietario
        dict_prop['partido'] = mapping_parties[name]
        dict_prop['link_curricula'] = link
        
        
        dict_sup = {}
        dict_sup['legislatura'] = 63
        dict_sup['id_legislador'] = str(uuid.uuid1())
        dict_sup['nombre_completo'] = suplente_propietario
        dict_sup['tipo_legislador'] = driver.find_element_by_css_selector('body > table:nth-child(1) > tbody > tr:nth-child(3) > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(2) > td:nth-child(2) > strong > font').text
        dict_sup['entidad'] = driver.find_element_by_css_selector('body > table:nth-child(1) > tbody > tr:nth-child(3) > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(3) > td:nth-child(2) > strong:nth-child(1) > font').text
        dict_sup['distrito'] = driver.find_element_by_css_selector('body > table:nth-child(1) > tbody > tr:nth-child(3) > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(3) > td:nth-child(2) > strong:nth-child(4) > font').text
        dict_sup['curul'] = driver.find_element_by_css_selector('body > table:nth-child(1) > tbody > tr:nth-child(3) > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(3) > td:nth-child(2) > strong:nth-child(7) > font').text
        dict_sup['suplente_propietario'] = 'S'
        dict_sup['name_suplente'] = ''
        dict_sup['partido'] = mapping_parties[name]
        dict_sup['link_curricula'] = link
        
        return([dict_prop, dict_sup])

## ID Data

In [319]:
data_id64 = []
for x, y in tqdm(zip(links_diputados, names_diputados), total = len(names_diputados)):
    driver.get(x)
    dicts = obtain_id_data(y, mapping_name_party, x)
    for j in dicts:
        data_id64.append(j)

100%|██████████| 500/500 [09:48<00:00,  1.18s/it]


In [320]:
with open('/Users/cblanesg/cam.blanes Dropbox/Camila Blanes/floor-speechMX/data/diputados/id_data/legislatura64.json', 'w') as json_file:
    json.dump(data_id64, json_file)

In [323]:
names = []
for i in data_id64:
    if i['nombre_completo'].lower() not in names:
        names.append(i['nombre_completo'].lower())

In [324]:
names

['aguilar castillo heriberto marcelo',
 'ramon angel valdez mendoza',
 'aguilar linda bonifacio',
 'feliciano romero martinez',
 'aguilar molina leticia arlett',
 'liliana garcia albores',
 'aguilera brenes sebastián',
 'jorge ariff marave sanchez',
 'aguilera chairez maribel',
 'martha alicia arreola martinez',
 'alfonso ramirez cuellar',
 'aguirre valenciana javier uriel',
 'alavez ruiz aleida',
 'maria concepcion franco rodriguez',
 'alegre salazar luis javier',
 'wilbert alberto batun chulim',
 'alejandro candelaria maximino',
 'luis alfredo ortiz pantoja',
 'alfaro morales maría isabel',
 'ma. guadalupe loredo gonzales',
 'almazán burgos karla yuritzi',
 'maria imelda agustina yescas ruiz',
 'alvarado moreno maría guillermina',
 'rebeca lidia garcia vallejo',
 'ambrocio gachuz josé guadalupe',
 'oscar mejia garcia',
 'andazola gómez socorro irma',
 'carlota lara guzman',
 'andrade zavala marco antonio',
 'daniel castillo de la cruz',
 'angeles mendoza julio césar',
 'luis eduardo 

In [178]:
mapping_id_cand = {}
for i in data_id:
    mapping_id_cand[i['nombre_completo']] = i['id_legislador']

In [136]:
pd.DataFrame(data_id).to_csv('/Users/cblanesg/cam.blanes Dropbox/Camila Blanes/floor-speechMX/data/diputados/id_data/id_data.csv', encoding = 'utf-8')

In [141]:
df_mapping_id_cand = pd.DataFrame()
df_mapping_id_cand['name'] = list(mapping_id_cand.keys())
df_mapping_id_cand['id'] = list(mapping_id_cand.values())

In [144]:
df_mapping_id_cand.to_csv('/Users/cblanesg/cam.blanes Dropbox/Camila Blanes/floor-speechMX/data/diputados/id_data/mapping_id_incumbent.csv', encoding = 'utf-8')

In [132]:
import json

In [179]:
with open('/Users/cblanesg/cam.blanes Dropbox/Camila Blanes/floor-speechMX/data/diputados/id_data/legislatura64.json', 'w') as json_file:
    json.dump(data_id, json_file)

## Comisiones

In [112]:
all_comisiones = []
for x, y in tqdm(zip(links_diputados, names_diputados), total = len(names_diputados)):
    driver.get(x)
    id_inc = mapping_id_cand[y]
    
    comisiones = driver.find_element_by_css_selector('body > div > table:nth-child(7) > tbody > tr > td > table:nth-child(1) > tbody > tr:nth-child(2) > td').text
    for i in re.sub('ORDINARIA', '', comisiones.split('\n')[0]).split('|'):
        if any(i):
            temp = {}
            temp['nombre_completo'] = y
            temp['id_legislador'] = id_inc
            temp['comision'] = re.sub('[ ]+$', '', re.sub('^[ ]+', '', i))
            all_comisiones.append(temp)
        else:
            continue

100%|██████████| 500/500 [10:39<00:00,  1.28s/it]


In [120]:
def dummy_remove(x):
    if any(re.findall('GRUPO DE AMISTAD', x)):
        return(1)
    else:
        return(0)
df = pd.DataFrame(all_comisiones)
df['dummy_remove'] = df.apply(lambda x: dummy_remove(x['comision']), axis = 1)

In [123]:
df_comisiones = df[df['dummy_remove'] == 0]
df_comisiones = df_comisiones[['nombre_completo', 'id_legislador', 'comision']]

In [125]:
df_comisiones

## Legislatura 63

In [248]:
main_url = 'http://sitllxiii.diputados.gob.mx/listado_diputados_gpnp.php?tipot=TOTAL'
driver.get(main_url)

In [249]:
links_diputados = []
names_diputados = []
for i in tqdm(driver.find_elements_by_css_selector('a')):
    link = i.get_attribute('href')
    if any(re.findall('curricula', link)):
        links_diputados.append(link)
        names_diputados.append(re.sub('^[0-9]+ ', '', i.text))
        
    else:
        continue

100%|██████████| 511/511 [00:12<00:00, 40.26it/s]


In [250]:
mapping_name_party = {}
for i in names_diputados[0:202]:
    mapping_name_party[i] = 'pri'
    
for i in names_diputados[202: 309]:
    mapping_name_party[i] = 'pan'
    
for i in names_diputados[309:360]:
    mapping_name_party[i] = 'prd'
    
for i in names_diputados[360:410]:
    mapping_name_party[i] = 'morena'
    
for i in names_diputados[410:448]:
    mapping_name_party[i] = 'partido verde ecologista'
    
for i in names_diputados[448:469]:
    mapping_name_party[i] = 'movimiento ciudadano'
    
for i in names_diputados[469:472]:
    mapping_name_party[i] = 'alianza'
    
for i in names_diputados[472:483]:
    mapping_name_party[i] = 'encuentro social'

for i in names_diputados[483:]:
    mapping_name_party[i] = 'sin partido'

In [251]:
len(mapping_name_party)

500

In [264]:
re.sub('[ ]+$', '', re.sub('[(]ficha curricular[)]', '', 'Pablo Basáñez García  (ficha curricular)'))

'Pablo Basáñez García'

In [283]:
def obtain_id_data63(name, mapping_parties, link):
    suplente_propietario = driver.find_element_by_css_selector('body > table:nth-child(1) > tbody > tr:nth-child(3) > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(6) > td:nth-child(2) > strong > font').text
    if any(re.findall('ficha curricular', suplente_propietario)):
        dict_prop = {}
        dict_prop['legislatura'] = 63
        dict_prop['id_legislador'] = str(uuid.uuid1())
        dict_prop['nombre_completo'] = re.sub('[ ]+$', '', re.sub('[(]ficha curricular[)]', '', suplente_propietario))
        dict_prop['tipo_legislador'] = driver.find_element_by_css_selector('body > table:nth-child(1) > tbody > tr:nth-child(3) > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(2) > td:nth-child(2) > strong > font').text
        dict_prop['entidad'] = driver.find_element_by_css_selector('body > table:nth-child(1) > tbody > tr:nth-child(3) > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(3) > td:nth-child(2) > strong:nth-child(1) > font').text
        dict_prop['distrito'] = driver.find_element_by_css_selector('body > table:nth-child(1) > tbody > tr:nth-child(3) > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(3) > td:nth-child(2) > strong:nth-child(4) > font').text
        dict_prop['curul'] = driver.find_element_by_css_selector('body > table:nth-child(1) > tbody > tr:nth-child(3) > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(3) > td:nth-child(2) > strong:nth-child(7) > font').text
        dict_prop['suplente_propietario'] = 'P'
        dict_prop['name_suplente'] = name
        dict_prop['partido'] = mapping_parties[name]
        dict_prop['link_curricula'] = link
        
        
        dict_sup = {}
        dict_sup['legislatura'] = 63
        dict_sup['id_legislador'] = str(uuid.uuid1())
        dict_sup['nombre_completo'] = name
        dict_sup['tipo_legislador'] = driver.find_element_by_css_selector('body > table:nth-child(1) > tbody > tr:nth-child(3) > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(2) > td:nth-child(2) > strong > font').text
        dict_sup['entidad'] = driver.find_element_by_css_selector('body > table:nth-child(1) > tbody > tr:nth-child(3) > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(3) > td:nth-child(2) > strong:nth-child(1) > font').text
        dict_sup['distrito'] = driver.find_element_by_css_selector('body > table:nth-child(1) > tbody > tr:nth-child(3) > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(3) > td:nth-child(2) > strong:nth-child(4) > font').text
        dict_sup['curul'] = driver.find_element_by_css_selector('body > table:nth-child(1) > tbody > tr:nth-child(3) > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(3) > td:nth-child(2) > strong:nth-child(7) > font').text
        dict_sup['suplente_propietario'] = 'S'
        dict_sup['name_suplente'] = ''
        dict_sup['partido'] = mapping_parties[name]
        dict_sup['link_curricula'] = link
        return([dict_prop, dict_sup])
    else:
    
        dict_prop = {}
        dict_prop['legislatura'] = 63
        dict_prop['id_legislador'] = str(uuid.uuid1())
        dict_prop['nombre_completo'] = name
        dict_prop['tipo_legislador'] = driver.find_element_by_css_selector('body > table:nth-child(1) > tbody > tr:nth-child(3) > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(2) > td:nth-child(2) > strong > font').text
        dict_prop['entidad'] = driver.find_element_by_css_selector('body > table:nth-child(1) > tbody > tr:nth-child(3) > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(3) > td:nth-child(2) > strong:nth-child(1) > font').text
        dict_prop['distrito'] = driver.find_element_by_css_selector('body > table:nth-child(1) > tbody > tr:nth-child(3) > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(3) > td:nth-child(2) > strong:nth-child(4) > font').text
        dict_prop['curul'] = driver.find_element_by_css_selector('body > table:nth-child(1) > tbody > tr:nth-child(3) > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(3) > td:nth-child(2) > strong:nth-child(7) > font').text
        dict_prop['suplente_propietario'] = 'P'
        dict_prop['name_suplente'] = suplente_propietario
        dict_prop['partido'] = mapping_parties[name]
        dict_prop['link_curricula'] = link
        
        
        dict_sup = {}
        dict_sup['legislatura'] = 63
        dict_sup['id_legislador'] = str(uuid.uuid1())
        dict_sup['nombre_completo'] = suplente_propietario
        dict_sup['tipo_legislador'] = driver.find_element_by_css_selector('body > table:nth-child(1) > tbody > tr:nth-child(3) > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(2) > td:nth-child(2) > strong > font').text
        dict_sup['entidad'] = driver.find_element_by_css_selector('body > table:nth-child(1) > tbody > tr:nth-child(3) > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(3) > td:nth-child(2) > strong:nth-child(1) > font').text
        dict_sup['distrito'] = driver.find_element_by_css_selector('body > table:nth-child(1) > tbody > tr:nth-child(3) > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(3) > td:nth-child(2) > strong:nth-child(4) > font').text
        dict_sup['curul'] = driver.find_element_by_css_selector('body > table:nth-child(1) > tbody > tr:nth-child(3) > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(3) > td:nth-child(2) > strong:nth-child(7) > font').text
        dict_sup['suplente_propietario'] = 'S'
        dict_sup['name_suplente'] = ''
        dict_sup['partido'] = mapping_parties[name]
        dict_sup['link_curricula'] = link
        
        return([dict_prop, dict_sup])

## ID Data

In [284]:
data_id = []
for x, y in tqdm(zip(links_diputados, names_diputados), total = len(names_diputados)):
    driver.get(x)
    dicts = obtain_id_data63(y, mapping_name_party, x)
    for i in  dicts:
        data_id.append(i)

100%|██████████| 500/500 [03:42<00:00,  2.25it/s]


In [285]:
len(data_id)

1000

In [286]:
mapping_id_cand = {}
for i in data_id:
    mapping_id_cand[i['nombre_completo']] = i['id_legislador']

In [287]:
import unidecode

In [328]:
len(data_id)

1000

In [330]:
dict_yunes = {}
dict_yunes['legislatura'] = 63
dict_yunes['id_legislador']= uuid.uuid1()
dict_yunes['nombre_completo']= 'Yunes Linares Miguel Angel'
dict_yunes['tipo_legislador']= 'Representacion Proporcional'
dict_yunes['entidad']= 'Veracruz'
dict_yunes['distrito']= 3
dict_yunes['curul']= ''
dict_yunes['suplente_propietario']=  'P'
dict_yunes['name_suplente']= 'Pérez Rodríguez Enrique'
dict_yunes['partido']= 'PAN'
dict_yunes['link_curricula']= ''

In [288]:
for i in data_id:
    i['nombre_completo'] = unidecode.unidecode(i['nombre_completo'])
    i['tipo_legislador'] = unidecode.unidecode(i['tipo_legislador'])
    i['name_suplente'] = unidecode.unidecode(i['name_suplente'])
    i['entidad'] = unidecode.unidecode(i['entidad'])

In [289]:
with open('/Users/cblanesg/cam.blanes Dropbox/Camila Blanes/floor-speechMX/data/diputados/id_data/legislatura63.json', 'w') as json_file:
    json.dump(data_id, json_file)

In [325]:
all_names = []
for i in data_id:
    if i['name_suplente'] not in all_names:
        all_names.append(i['name_suplente'].lower())

In [326]:
all_names

['claudia janeth ochoa iniguez',
 '',
 'carlos joaquin sosa lopez',
 'jesus antonio hazouri mansur',
 'raymundo moreno ibanez',
 'ma. guadalupe calderon medina',
 'marco antonio barranco sanchez',
 'alonso que erubiel lorenzo',
 'sinthia del carmen alvarado canche',
 'fabiola rosas cuautle',
 'pablo casas jaime',
 'elizabeth hernandez calderon',
 'eduardo herrera calixto',
 'veronica boreque martinez gonzalez',
 'jose lucas vallarta chan',
 'juana leticia hernandez armas',
 'balcazar narro rafael arturo',
 'rogelio lopez angulo',
 'ana yolanda esquivel jaramillo',
 'cesar alfonso padilla vazquez',
 'carlos rangel aquino',
 'jose luis lima gonzalez',
 'maria esther fragoso teran',
 'laura beatriz hernandez tapia',
 'bojorquez castillo martha julisa',
 'santos garza herrera',
 'borunda espinoza brenda',
 'yolanda eugenia gonzalez hernandez',
 'brito herrera gonzalo rene',
 'rubi alejandra cardoso guzman',
 'renato dorantes dorantes',
 'magin pereda martinez',
 'gustavo sotelo villegas',
